In [ ]:
"""
Just a prototype to use landsat data from earth engine to get the surface thermal tempereture at
the weather station to verify that this works correctly
"""

In [ ]:
#pip install geemap

     |████████████████████████████████| 2.0 MB 13.2 MB/s 
     |████████████████████████████████| 130 kB 70.7 MB/s 
     |████████████████████████████████| 95 kB 3.3 MB/s 
     |████████████████████████████████| 99 kB 7.5 MB/s 
     |████████████████████████████████| 44 kB 2.7 MB/s 
     |████████████████████████████████| 1.2 MB 37.2 MB/s 
     |████████████████████████████████| 98 kB 6.7 MB/s 
     |████████████████████████████████| 187 kB 61.1 MB/s 
     |████████████████████████████████| 1.3 MB 34.1 MB/s 
     |████████████████████████████████| 3.0 MB 62.2 MB/s 
     |████████████████████████████████| 8.7 MB 52.7 MB/s 
     |████████████████████████████████| 3.3 MB 54.3 MB/s 
     |████████████████████████████████| 128 kB 47.7 MB/s 
     |████████████████████████████████| 342 kB 60.1 MB/s 
     |████████████████████████████████| 428 kB 69.4 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()
import geemap

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=SJB9FlSlM6pXvCqccPu4T2P4UCphHFIUXrvMWTkuAfI&tc=Y8hQFFl5Kar-ZgJKNcgIGsggNDBOM32CfxF3TCkdfCY&cc=ly-_NI99oXSYHduLaRkiA0D_Kh9zuCAubOG0tuNbfqs

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWj7iW86-VU21Ot-IIl0pSnNcr7afPtNcieqhF-9l2v8K-MweJ0Xgy4

Successfully saved authorization token.


In [30]:
#location of wheather station 
#lon/lat
mgtLoc = ee.Geometry.Point([165.01,-80.00])
#location of another random place
#mgtLoc = ee.Geometry.Point([117.35,29.736])

res=100 #thermal resolution

In [31]:
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate('2015-06-01', '2016-06-01').filterBounds(
    mgtLoc)

print(collection.size().getInfo())

0


In [ ]:
#visualize the one image that I apaprently have

image=collection.first()

def apply_scale_factors(image,thermalBand):
    #as per the data sets description here https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2
    thermalBands = image.select(thermalBand).multiply(0.00341802).add(149.0)
    return image.addBands(thermalBands, None, True)


vis_thermal={
    'bands':["ST_B10"]
}

dataset = apply_scale_factors(image,vis_thermal['bands'])



In [ ]:
Map = geemap.Map()
Map.addLayer(dataset, vis_thermal, 'Color infrared (543)')
Map.centerObject(mgtLoc, 13)
Map

Map(center=[29.736, 117.35000000000001], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

In [ ]:
#get the surface tempereture on a particular day

image.sample(xy, res).first().get('elevation').getInfo()

Find out how many Landsat imgaes are available.

In [ ]:
#location of wheather station 
#lon/lat
mgtLoc = ee.Geometry.Point([-80.00, 165.01])
res=100 #thermal resolution


Create a median composite.

In [ ]:
median = collection.median() #gets the median pixel value for all of the images in this collection, easy to visualize 

Scale per https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands

In [ ]:
def apply_scale_factors(image,visBands,visNIR):
    #as per the data sets description here https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T2_SR#bands
    opticalBands = image.select(visBands).multiply(0.0001)
    thermalBands = image.select(visNIR).multiply(0.0001)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

In [ ]:
dataset = apply_scale_factors(median,vis_natural['bands'],vis_nir['bands'])

Specify visualization parameters.

Add data layers to the map.

In [ ]:
Map.addLayer(dataset, vis_natural, 'True color (432)')
Map.addLayer(dataset, vis_nir, 'Color infrared (543)')
Map

In [ ]:
### now try sentinel data
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2
s2 = ee.ImageCollection("COPERNICUS/S2")
print(collection.size().getInfo())
medianS2=s2.median()

In [ ]:
def apply_scale_factors(image,visBands,visNIR):
    #as per the data sets description here https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T2_SR#bands
    opticalBands = image.select(visBands).multiply(0.0001)
    thermalBands = image.select(visNIR).multiply(0.0001)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

In [ ]:
vis_natural = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.0,
    'max': 0.3,
}

vis_nir = {
    'bands': ['B6','B7','B8'],
    'min': 0.0,
    'max': 0.3,
}

In [ ]:
s2Dataset = apply_scale_factors(medianS2,vis_natural['bands'],vis_nir['bands'])

In [ ]:
Map.addLayer(s2Dataset, vis_nir, 'Color infrared (543)')
Map